<a href="https://colab.research.google.com/github/Mingjie-Shen/Deeep-Learning-Homework-and-Practice/blob/main/Solution_Q2PS1_byTutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

import h5py
import time
import copy
from random import randint

#load MNIST data
MNIST_data = h5py.File('MNISTdata.hdf5', 'r')
x_train = np.float32(MNIST_data['x_train'][:] )
y_train = np.int32(np.array(MNIST_data['y_train'][:,0]))
x_test = np.float32( MNIST_data['x_test'][:] )
y_test = np.int32( np.array( MNIST_data['y_test'][:,0]  ) )

MNIST_data.close()



####################################################################################

#Implementation of backpropagation algorithm



#number of hidden units
H = 200
#number of inputs
num_inputs = 28*28
#number of outputs
num_outputs = 10


model = {}
model['W1'] = np.random.randn(H,num_inputs) / np.sqrt(num_inputs) 
model['W2'] = np.random.randn(num_outputs, H) / np.sqrt(H)


#no biases here!

def hidden_function(z):
    ZZ = np.exp(z)/(1+np.exp(z))
    return ZZ

def softmax_function(z):
    ZZ = np.exp(z)/np.sum(np.exp(z))
    return ZZ

def hidden_function_grad(z):
    ZZ = np.divide( np.exp(z),  (1+np.exp(z) ) )  - np.divide( np.exp(2*z),   np.multiply( (1+np.exp(z)), (1+np.exp(z)) ) )
    #ZZ = np.array(ZZ[0][0])
    return ZZ

def forward(x,y, model):
    Z1 =  np.dot(model['W1'], x)
    A = hidden_function(Z1)
    A_grad = hidden_function_grad(Z1)
    Z2 = np.dot(model['W2'], A.T)
    p = softmax_function(Z2)
    return p, A, A_grad


def backward(x,y, A, A_grad, model, delta3):
    W2_grad = np.dot(np.matrix(A).T, np.matrix(delta3))
    delta2 = np.zeros(H)
    for i in range(H):
        delta2[i] = np.dot(model['W2'][:,i].T, delta3 )*A_grad[i]
    W1_grad = np.dot(np.matrix(x).T, np.matrix(delta2) )
    return np.array(W1_grad), np.array(W2_grad)
    
    
import time

time1 = time.time()
LR = .01
num_epochs = 2

for epochs in range(num_epochs):
    #len(train)
    total_correct = 0
    for n in range( len(x_train)):
        n_random = randint(0,len(x_train)-1 )
        y = y_train[n_random]
        x = x_train[n_random][:]
        p, A, A_grad = forward(x, y, model)
        prediction = np.argmax(p)
        if (prediction == y):
            total_correct += 1
        delta3 = -1.0*p
        delta3[y] += 1
        W1_grad, W2_grad = backward(x,y, A, A_grad, model, delta3)
        model['W2'] = model['W2'] + LR*W2_grad.T
        model['W1'] = model['W1'] + LR*W1_grad.T
    print(total_correct/np.float(len(x_train) ) )

time2 = time.time()
print(time2-time1)
        
    
######################################################
#test data
total_correct = 0
for n in range( len(x_test)):
    y = y_test[n]
    x = x_test[n][:]
    p, A, A_grad = forward(x, y, model)
    prediction = np.argmax(p)
    if (prediction == y):
        total_correct += 1
        
print(total_correct/np.float(len(x_test) ) )